In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import data and prepare DFs

In [2]:
train_no_label = pd.read_csv('../../data/artificial_train.data', sep=' ', header=None)
train_label = pd.read_csv('../../data/artificial_train.labels', sep=' ', header=None)
test_no_label = pd.read_csv('../../data/artificial_test.data', sep=' ', header=None)

In [3]:
# change the label to 0 and 1
train_label = train_label.replace(-1, 0)

In [4]:
#drop NaN columns
train_no_label = train_no_label.dropna(axis=1)
test_no_label = test_no_label.dropna(axis=1)

In [5]:
train_no_label_columns = ["c" + str(i) for i in range(1, len(train_no_label.columns) + 1)]
train_no_label.columns = train_no_label_columns

test_no_label_columns = ["c" + str(i) for i in range(1, len(test_no_label.columns) + 1)]
test_no_label.columns = test_no_label_columns

In [6]:
train = pd.concat([train_no_label, train_label], axis=1)
train.rename(columns={0: 'label'}, inplace=True)

test_no_label.rename(columns={0: 'label'}, inplace=True)

In [7]:
import copy as cp
test = cp.deepcopy(test_no_label)

In [8]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, test_size=0.2, random_state=42, stratify=train['label'])

In [9]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='label', path='agModels-predictClass', eval_metric='balanced_accuracy')  # specifies folder where to store trained models

predictor.fit(train_data=train, tuning_data=val, time_limit=600)

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
predictor.leaderboard(val, extra_metrics=['balanced_accuracy'])

                  model  score_test  balanced_accuracy  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2      0.8525             0.8525     0.8525        0.021320       0.012548  15.792172                 0.001086                0.000503           0.215811            2       True         13
1              CatBoost      0.8475             0.8475     0.8475        0.011685       0.006388   2.699897                 0.011685                0.006388           2.699897            1       True          7
2              LightGBM      0.8350             0.8350     0.8350        0.004221       0.002571   2.757311                 0.004221                0.002571           2.757311            1       True          4
3         LightGBMLarge      0.8350             0.8350     0.8350        0.004328       0.003086  10.119153                 0.004328                0.003086

,model,score_test,balanced_accuracy,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.8525,0.8525,0.8525,0.021320,0.012548,15.792172,0.001086,0.000503,0.215811,2,True,13
1,CatBoost,0.8475,0.8475,0.8475,0.011685,0.006388,2.699897,0.011685,0.006388,2.699897,1,True,7
2,LightGBM,0.8350,0.8350,0.8350,0.004221,0.002571,2.757311,0.004221,0.002571,2.757311,1,True,4
3,LightGBMLarge,0.8350,0.8350,0.8350,0.004328,0.003086,10.119153,0.004328,0.003086,10.119153,1,True,12
4,LightGBMXT,0.7700,0.7700,0.7700,0.003974,0.002864,2.690379,0.003974,0.002864,2.690379,1,True,3
5,KNeighborsUnif,0.7225,0.7225,0.7225,0.008877,0.048465,2.826612,0.008877,0.048465,2.826612,1,True,1
6,KNeighborsDist,0.7225,0.7225,0.7225,0.010491,0.013672,0.026149,0.010491,0.013672,0.026149,1,True,2
7,RandomForestEntr,0.7025,0.7025,0.7025,0.044954,0.038937,0.649433,0.044954,0.038937,0.649433,1,True,6
8,RandomForestGini,0.6525,0.6525,0.6525,0.047236,0.041746,0.775944,0.047236,0.041746,0.775944,1,True,5
9,ExtraTreesEntr,0.6425,0.6425,0.6425,0.050999,0.040431,0.362362,0.050999,0.040431,0.362362,1,True,9


In [ ]:
test_results = predictor.predict(test_no_label)
test_results


0      0
1      0
2      0
3      1
4      0
      ..
595    1
596    0
597    1
598    1
599    0
Name: label, Length: 600, dtype: int64

In [ ]:
# predict on test data with no label
test_results = predictor.predict_proba(test_no_label)[1]

test_results = pd.DataFrame(test_results)
test_results.rename(columns={0: '313420_313435'}, inplace=True)

In [ ]:
# save the results to a csv file
test_results.to_csv('313420_313435_artifical_automl_prediction.txt', index=False)